![](./images/Title.PNG)
<div class="alert alert-block alert-info"> <b> </b> 
</div>

# Fraud detection from transactions
![](./images/workflow_fraud.PNG)

<div class="alert alert-block alert-info"> <b> </b> 
</div>

## 1 - Connect to Vantage
<div class="alert alert-block alert-info"> <b> </b> 
</div>

![](./images/Slide32.PNG)

In [1]:
import teradataml as tdml
tdml.options.configure.byom_install_location = "mldb"
tdml.display.print_sqlmr_query = False
import getpass
import json
tdml.__version__
from datetime import datetime
tic = datetime.now()
tdml.options.display

<module 'teradataml.options.display' from 'C:\\Users\\dm250067\\Anaconda3\\envs\\vantage39\\lib\\site-packages\\teradataml\\options\\display.py'>

In [2]:
tdml.__version__

'17.20.00.04'

In [7]:
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import os

In [26]:
# set the path to the local project repository for this model demo
model_local_path = 'C:/Users/dm250067/OneDrive - Teradata/Documents/01 - Code Development/modelops-demo-models/model_definitions/transaction_fraud_indb'
res = os.system(f'mkdir -p "{model_local_path}"')
res = os.system(f'mkdir -p "{model_local_path}/model_modules"')

In [4]:
Param = {
    'host'          : 'tdprd2.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLSLSEMEA_DEMO_BANKING',
    'temp_database_name' : 'dm250067'
    }

Param = {
    'host'          : 'tdprd3.td.teradata.com', 
    'user'          : 'dm250067', 
    'password'      : "ENCRYPTED_PASSWORD(file:{},file:{})".format ('../../PassKey.properties','../../EncPass.properties'), #getpass.getpass(), 
    'logmech'       : 'LDAP',
    'database'      : 'ADLDSD_CHURN',
    'temp_database_name' : 'dm250067'
    }

tdml.create_context(**Param)

C:\Users\dm250067\Anaconda3\envs\vantage39\lib\site-packages\teradataml\context\context.py:480: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


Engine(teradatasql://:***@tdprd3.td.teradata.com/?DATABASE=ADLDSD_CHURN&LOGDATA=%2A%2A%2A&LOGMECH=%2A%2A%2A&USER=DM250067)

## 2 - Define Training Function

In [ ]:
%%writefile $model_local_path/model_modules/training.py
from teradataml import (
    DataFrame,
    GLM,
    ScaleFit,
    ScaleTransform,
    DecisionForest
)
from aoa import (
    record_training_stats,
    save_plot,
    aoa_create_context,
    ModelContext
)
import numpy as np


def plot_roc_curve(fi, img_filename):
    import pandas as pd
    import matplotlib.pyplot as plt
    feat_importances = pd.Series(fi)
    feat_importances.nlargest(10).plot(kind='barh').set_title('Feature Importance')
    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def train(context: ModelContext, **kwargs):
    aoa_create_context()

    feature_names = context.dataset_info.feature_names
    target_name   = context.dataset_info.target_names[0]
    entity_key    = context.dataset_info.entity_key

    # read training dataset from Teradata and convert to pandas
    train_df      = DataFrame.from_query(context.dataset_info.sql)

    print ("Scaling using InDB Functions...")
    
    scaler = ScaleFit(
        data           =train_df,
        target_columns = feature_names,
        scale_method   = context.hyperparams["scale_method"],
        miss_value     = context.hyperparams["miss_value"],
        global_scale   = context.hyperparams["global_scale"].lower() in ['true', '1'],
        multiplier     = context.hyperparams["multiplier"],
        intercept      = context.hyperparams["intercept"]
    )

    scaled_train = ScaleTransform(
        data           = train_df,
        object         = scaler.output,
        accumulate     = [target_name, entity_key]
    )
    
    scaler.output.to_sql(f"scaler_${context.model_version}", if_exists="replace")
    print("Saved scaler")
    
    print("Starting training...")
DecisionForest_obj = tdml.DecisionForest(data = ADS_scaled_training, 
                            input_columns = features, 
                            response_column = target, 
                            max_depth = 15, 
                            num_trees = 72, 
                            min_node_size = 1, 
                            mtry = 6, 
                            mtry_seed = 1, 
                            seed = 1, 
                            tree_type = 'CLASSIFICATION')
    model = DecisionForest(
        input_columns        = feature_names,
        response_column      = target_name,
        data                 = scaled_train.result,
        max_depth            = context.hyperparams["max_depth"],
        num_trees            = context.hyperparams["num_trees"],
        min_node_size        = context.hyperparams["min_node_size"],
        mtry                 = context.hyperparams["mtry"],
        mtry_seed            = context.hyperparams["mtry_seed"],
        seed                 = context.hyperparams["seed"],
        tree_type            = 'CLASSIFICATION'
    )
    
    model.result.to_sql(f"model_${context.model_version}", if_exists="replace")    
    print("Saved trained model")

    # Calculate feature importance and generate plot
    model_pdf = model.result.to_pandas()[['predictor','estimate']]
    predictor_dict = {}
    
    for index, row in model_pdf.iterrows():
        if row['predictor'] in feature_names:
            value = row['estimate']
            predictor_dict[row['predictor']] = value
    
    feature_importance = dict(sorted(predictor_dict.items(), key=lambda x: x[1], reverse=True))
    keys, values = zip(*feature_importance.items())
    norm_values = (values-np.min(values))/(np.max(values)-np.min(values))
    feature_importance = {keys[i]: float(norm_values[i]*1000) for i in range(len(keys))}
    plot_feature_importance(feature_importance, f"{context.artifact_output_path}/feature_importance")

    record_training_stats(
        train_df,
        features=feature_names,
        targets=[target_name],
        categorical=[target_name],
        feature_importance=feature_importance,
        context=context
    )